# Belegungsquote der Frauenhäuser in Deutschland

## Load source data and format

In [1]:
import pandas as pd
import numpy as np

fpath = "frauenhaus_suche.json"

# load data
raw = pd.read_json(fpath, orient="index")
raw["shelter_id"] = raw.index
raw = raw.reset_index(drop=True)

# clean up geometry columns
raw.loc[raw.geography.notnull(), "geometry"] = raw.loc[raw.geography.notnull(), "geography"]

# get keys
keys = pd.read_csv("data/helpers/free_places_key.csv")

# get all timeseries data
df = pd.DataFrame()

for i, row in raw.iterrows():
    
    temp = pd.DataFrame(row["data"])
    temp["shelter_id"] = row["shelter_id"]
    temp["shelter_name"] = row["title"]
    temp["latitude"] = row["geometry"]["coordinates"][1]
    temp["longitude"] = row["geometry"]["coordinates"][0]
    df = pd.concat([df, temp])
    
# reformat timeseries
df.timestamp = df.timestamp.apply(lambda x : pd.to_datetime(x, dayfirst=True))
df["date"] = df.timestamp.dt.date

# trim dates
df = df.loc[(df.date >= pd.to_datetime("2022-01-01")) & (df.date <= pd.to_datetime("2022-12-31")),]

# get monthyear as column
df["monthyear"] = df.timestamp.dt.to_period("M")

# add text keys
df.loc[df.freePlaces=="", "freePlaces"] = np.nan
df.freePlaces = df.freePlaces.astype(float)
df = pd.merge(df, keys, on="freePlaces")

# fill nas
df.loc[df.description.isnull(), "description"] = "k.A."

# add simplified NAs
desc_short = {
    "Aufnahme möglich für Frauen mit 4 oder mehr Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen mit 3 Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen mit 2 Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen ohne Kinder, Aufnahme möglich für Frauen mit 1 Kind":"Aufnahme möglich",
    "Aufnahme möglich für Frauen ohne Kinder":"Aufnahme möglich",
    "Keine Aufnahme möglich":"Keine Aufnahme möglich",
    "Aufnahme möglich ohne detaillierte Angabe":"Aufnahme möglich",
    "k.A.":"keine Angabe"
}

df["status"] = df.description.map(desc_short)

/Users/maxdonheiser/.pyenv/versions/3.9.1/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


## Calculate overall counts of days/timestamps to use as denominator

In [2]:
# calculate counts for dividing
n_timestamps = len(df.timestamp.unique())
n_days = len(df.date.unique())
n_months = len(df.monthyear.unique())

## Load in geodata and metadata

In [3]:
# add bundesland
geocoded = pd.read_csv("data/helpers/shelters_geocoded.csv").drop(["latitude","longitude"], axis=1)
geocoded["kreis"] = geocoded.apply(lambda row : f"{row.bez} {row.gen}", axis=1)
geocoded = geocoded[["shelter_id","bundesland","kreis"]]

In [4]:
# load metadata
metadata = pd.read_csv("data/helpers/shelters_metadata.csv")
metadata = metadata.drop(['title'], axis=1)

## Start list of Frauenhäuser that don't meet data standards
First step: filter out shelters that showed up in the data less than 80% of the time

In [5]:
data_completeness = df.groupby(["shelter_id"]).timestamp.count().reset_index()
data_completeness["pct_data_availability"] = data_completeness["timestamp"]/n_timestamps
to_drop = data_completeness.loc[data_completeness.pct_data_availability < .8, "shelter_id"].values

Second step: get all shelters with more than 20% keine Angabe (based on raw data, not on daily summary)

In [6]:
status_counts = df.groupby(["shelter_id", "status"]).agg(
    status_count = ("status","count")
).reset_index()
status_counts = status_counts.loc[status_counts.status=="keine Angabe",]
status_counts["pct"] = status_counts["status_count"]/n_timestamps
to_add = status_counts.loc[status_counts.pct > .20, "shelter_id"].values
to_drop = np.concatenate([to_drop, to_add])

Third step: Filter out entries that arent Frauenhäuser

In [7]:
to_add = metadata.loc[metadata.einrichtungsart != "Frauenhaus", "shelter_id"].values
to_drop = np.concatenate([to_drop, to_add])

Map to source data

In [8]:
df["to_include"] = df.shelter_id.map(lambda x : x not in to_drop)

## Reshape data as one value per day
3x keine Angabe = keine Angabe, mind. 1 aufnahme möglich = aufnahme möglich, sonst aufnahme nicht möglich

In [9]:
# get counts per day
df_long = df.loc[df.to_include==True,] \
    .groupby(["shelter_name","shelter_id","date","status"]).agg(
        status_count = ("status", "count")
    ).reset_index()

df_long.head()

,shelter_name,shelter_id,date,status,status_count
0,1. Autonomes Frauenhaus,2027,2022-01-01,Keine Aufnahme möglich,3
1,1. Autonomes Frauenhaus,2027,2022-01-02,Keine Aufnahme möglich,3
2,1. Autonomes Frauenhaus,2027,2022-01-03,Keine Aufnahme möglich,3
3,1. Autonomes Frauenhaus,2027,2022-01-04,Keine Aufnahme möglich,3
4,1. Autonomes Frauenhaus,2027,2022-01-05,Keine Aufnahme möglich,3


In [10]:
# long to wide
df_daily = pd.pivot(
    df_long,
    index=["shelter_name","shelter_id","date"],
    values="status_count",
    columns="status"
).reset_index().replace(np.nan, 0)

# calculate counts on day, sometimes scraper failed
df_daily["day_counts"] = df_daily["Aufnahme möglich"] + df_daily["Keine Aufnahme möglich"] + df_daily["keine Angabe"]

df_daily["status"] = "keine Aufnahme möglich"
df_daily.loc[df_daily["keine Angabe"] == df_daily["day_counts"], "status"] = "keine Angabe"
df_daily.loc[df_daily["Aufnahme möglich"] > 0, "status"] = "Aufnahme möglich"

df_daily.head()

status,shelter_name,shelter_id,date,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,day_counts,status
0,1. Autonomes Frauenhaus,2027,2022-01-01,0.0,3.0,0.0,3.0,keine Aufnahme möglich
1,1. Autonomes Frauenhaus,2027,2022-01-02,0.0,3.0,0.0,3.0,keine Aufnahme möglich
2,1. Autonomes Frauenhaus,2027,2022-01-03,0.0,3.0,0.0,3.0,keine Aufnahme möglich
3,1. Autonomes Frauenhaus,2027,2022-01-04,0.0,3.0,0.0,3.0,keine Aufnahme möglich
4,1. Autonomes Frauenhaus,2027,2022-01-05,0.0,3.0,0.0,3.0,keine Aufnahme möglich


## See overall shelter counts by bundesland

In [11]:
# calculate overall number of shelters we can use by bundesland
df = pd.merge(df, geocoded, on="shelter_id")
shelter_counts = df[["shelter_id","bundesland"]].drop_duplicates().groupby(["bundesland"]).agg(
    n_shelters = ("bundesland", "count")
).reset_index()
shelter_included = df.loc[df.to_include==True,["shelter_id","bundesland"]].drop_duplicates().groupby(["bundesland"]).agg(
    n_analyzed = ("bundesland", "count")
).reset_index()
shelter_counts = pd.merge(shelter_counts, shelter_included, how="outer").replace(np.nan, 0)
shelter_counts

,bundesland,n_shelters,n_analyzed
0,Baden-Württemberg,50,22.0
1,Bayern,42,19.0
2,Berlin,12,0.0
3,Brandenburg,10,4.0
4,Bremen,3,0.0
5,Hamburg,5,0.0
6,Hessen,27,27.0
7,Mecklenburg-Vorpommern,10,6.0
8,Niedersachsen,28,10.0
9,Nordrhein-Westfalen,62,62.0


## Overall shelter-level summary

In [12]:
df_shelter = df_daily.groupby(["shelter_name","shelter_id","status"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter["pct"] = df_shelter["n"]/n_days
df_shelter

,shelter_name,shelter_id,status,n,pct
0,1. Autonomes Frauenhaus,2027,Aufnahme möglich,290,0.794521
1,1. Autonomes Frauenhaus,2027,keine Aufnahme möglich,75,0.205479
2,1. Frauenhaus Köln,2140,Aufnahme möglich,9,0.024658
3,1. Frauenhaus Köln,2140,keine Aufnahme möglich,356,0.975342
4,2. Autonomes Frauenhaus Köln,2253,Aufnahme möglich,17,0.046575
...,...,...,...,...,...
395,Städtisches Frauenhaus Stuttgart,2197,Aufnahme möglich,130,0.356164
396,Städtisches Frauenhaus Stuttgart,2197,keine Angabe,49,0.134247
397,Städtisches Frauenhaus Stuttgart,2197,keine Aufnahme möglich,186,0.509589
398,Wolfsburger Frauenhaus,2383,Aufnahme möglich,101,0.276712


In [13]:
# pivot to wide and fill values
df_shelter_wide = pd.pivot(
    df_shelter,
    index=["shelter_name","shelter_id"],
    columns="status",
    values="pct"
).reset_index().replace(np.nan, 0)

# drop bad ids
df_shelter_wide = df_shelter_wide.loc[~df_shelter_wide.shelter_id.isin(to_drop),]

# and add geocoding
df_shelter_wide = pd.merge(df_shelter_wide, geocoded, how="left")
df_shelter_wide.head()

,shelter_name,shelter_id,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,bundesland,kreis
0,1. Autonomes Frauenhaus,2027,0.794521,0.0,0.205479,Sachsen,Kreisfreie Stadt Leipzig
1,1. Frauenhaus Köln,2140,0.024658,0.0,0.975342,Nordrhein-Westfalen,Kreisfreie Stadt Köln
2,2. Autonomes Frauenhaus Köln,2253,0.046575,0.0,0.953425,Nordrhein-Westfalen,Kreisfreie Stadt Köln
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,0.378082,0.0,0.621918,Sachsen,Kreisfreie Stadt Leipzig
4,AWO Frauen- und Kinderschutzhaus Bodenseekreis,2333,0.000000,0.0,1.000000,Baden-Württemberg,Kreis Bodenseekreis


In [14]:
# drop shelter id
df_shelter_wide = df_shelter_wide.drop(["shelter_id"], axis=1)

In [15]:
# filter out shelters data and save to csv
df_shelter_wide.to_csv("./data/cleaned/data_by_shelter.csv", index=False)

## Shelter-level data by month

In [16]:
# get monthyear
df_daily["monthyear"] = pd.to_datetime(df_daily.date).dt.to_period("M")

In [17]:
# get count of values per day
df_monthyear = df_daily.groupby(["shelter_name","shelter_id","monthyear","status"]).agg(
    status_count = ("status", "count")
).reset_index()
df_monthyear = df_monthyear.loc[~df_monthyear.shelter_id.isin(to_drop),] # filter out incomplete, dont need to share with lokalen
df_monthyear.head()

,shelter_name,shelter_id,monthyear,status,status_count
0,1. Autonomes Frauenhaus,2027,2022-01,Aufnahme möglich,1
1,1. Autonomes Frauenhaus,2027,2022-01,keine Aufnahme möglich,30
2,1. Autonomes Frauenhaus,2027,2022-02,Aufnahme möglich,28
3,1. Autonomes Frauenhaus,2027,2022-03,Aufnahme möglich,31
4,1. Autonomes Frauenhaus,2027,2022-04,Aufnahme möglich,30


In [18]:
# pivot to wide and fill values
df_monthyear_wide = pd.pivot(
    df_monthyear,
    index=["shelter_name","shelter_id","monthyear"],
    columns="status",
    values="status_count"
).reset_index().replace(np.nan, 0)
df_monthyear_wide.head()

status,shelter_name,shelter_id,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich
0,1. Autonomes Frauenhaus,2027,2022-01,1.0,0.0,30.0
1,1. Autonomes Frauenhaus,2027,2022-02,28.0,0.0,0.0
2,1. Autonomes Frauenhaus,2027,2022-03,31.0,0.0,0.0
3,1. Autonomes Frauenhaus,2027,2022-04,30.0,0.0,0.0
4,1. Autonomes Frauenhaus,2027,2022-05,31.0,0.0,0.0


In [19]:
# get days in each month
day_counts = df_daily[["monthyear","date"]].drop_duplicates().groupby(["monthyear"]).agg(
    timestamp_count = ("date", "count")
).reset_index()

In [20]:
# add total count of timestamp for percent denominator
df_monthyear_wide = pd.merge(df_monthyear_wide, day_counts, how="outer")
df_monthyear_wide.head()

,shelter_name,shelter_id,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,timestamp_count
0,1. Autonomes Frauenhaus,2027,2022-01,1.0,0.0,30.0,31
1,1. Frauenhaus Köln,2140,2022-01,4.0,0.0,27.0,31
2,2. Autonomes Frauenhaus Köln,2253,2022-01,4.0,0.0,27.0,31
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,2022-01,31.0,0.0,0.0,31
4,AWO Frauen- und Kinderschutzhaus Bodenseekreis,2333,2022-01,0.0,0.0,31.0,31


In [21]:
# calculate percentages
df_monthyear_wide.iloc[:,-4:-1] = df_monthyear_wide.iloc[:,-4:-1].apply(lambda x : x / df_monthyear_wide.timestamp_count, axis=0)
df_monthyear_wide.head()

,shelter_name,shelter_id,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,timestamp_count
0,1. Autonomes Frauenhaus,2027,2022-01,0.032258,0.0,0.967742,31
1,1. Frauenhaus Köln,2140,2022-01,0.129032,0.0,0.870968,31
2,2. Autonomes Frauenhaus Köln,2253,2022-01,0.129032,0.0,0.870968,31
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,2022-01,1.000000,0.0,0.000000,31
4,AWO Frauen- und Kinderschutzhaus Bodenseekreis,2333,2022-01,0.000000,0.0,1.000000,31


In [22]:
# merge with geodata
df_monthyear_wide = pd.merge(df_monthyear_wide, geocoded, on="shelter_id", how="left")

In [23]:
# drop timestamp camp and save file
df_monthyear_wide = df_monthyear_wide.drop(["timestamp_count", "shelter_id"], axis=1)

In [24]:
df_monthyear_wide.to_csv("./data/cleaned/data_by_month_and_shelter.csv", index=False)

## Bundesland-level summary

In [25]:
df_daily_geocoded = pd.merge(df_daily, geocoded)

In [26]:
bundesland_counts = df_daily_geocoded.groupby(["bundesland"]).agg(
    n = ("bundesland", "count")
).reset_index()

In [27]:
bundesland = df_daily_geocoded.groupby(["bundesland", "status"]).agg(
    status_count = ("status", "count")
).reset_index()

bundesland = pd.merge(bundesland, bundesland_counts, on="bundesland")
bundesland["pct"] = bundesland["status_count"]/bundesland["n"]

bundesland_wide = pd.pivot(bundesland, index="bundesland", columns="status", values="pct").reset_index()

bundesland_wide = pd.merge(bundesland_wide, shelter_counts, on="bundesland", how="outer").replace(np.nan, 0)

bundesland_wide.head()

,bundesland,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,n_shelters,n_analyzed
0,Baden-Württemberg,0.139823,0.029962,0.830215,50,22.0
1,Bayern,0.171017,0.014275,0.814708,42,19.0
2,Brandenburg,0.308219,0.036301,0.655479,10,4.0
3,Hessen,0.084035,0.000000,0.915965,27,27.0
4,Mecklenburg-Vorpommern,0.231718,0.021481,0.746801,10,6.0


In [28]:
bundesland_wide.to_csv("./data/cleaned/bundesland_overview.csv", index=False)

## Bundesland and month-level summary

In [29]:
bundesland_counts = df_daily_geocoded.groupby(["bundesland","monthyear"]).agg(
    n = ("bundesland", "count")
).reset_index()

In [30]:
bundesland = df_daily_geocoded.groupby(["bundesland", "monthyear", "status"]).agg(
    status_count = ("status", "count")
).reset_index()

bundesland = pd.merge(bundesland, bundesland_counts, on=["bundesland","monthyear"])
bundesland["pct"] = bundesland["status_count"]/bundesland["n"]

bundesland_wide = pd.pivot(bundesland, index=["bundesland","monthyear"], columns="status", values="pct").reset_index()

bundesland_wide = pd.merge(bundesland_wide, shelter_counts, on="bundesland", how="outer").replace(np.nan, 0)

bundesland_wide.head()

,bundesland,monthyear,Aufnahme möglich,keine Angabe,keine Aufnahme möglich,n_shelters,n_analyzed
0,Baden-Württemberg,2022-01,0.273425,0.018433,0.708141,50,22.0
1,Baden-Württemberg,2022-02,0.253401,0.000000,0.746599,50,22.0
2,Baden-Württemberg,2022-03,0.124438,0.037481,0.838081,50,22.0
3,Baden-Württemberg,2022-04,0.130303,0.059091,0.810606,50,22.0
4,Baden-Württemberg,2022-05,0.143695,0.039589,0.816716,50,22.0


In [31]:
bundesland_wide.to_csv("./data/cleaned/monthly_bundesland_overview.csv", index=False)

## Weihnachts Beispiel - 12.25.2022 um 8 Uhr

In [32]:
# filter by data and get relevant variables
weihnacht = df.loc[df.timestamp==pd.to_datetime("2022-25-12 08:01:00", dayfirst=True),] # filter by day
weihnacht = weihnacht[["shelter_name","shelter_id","latitude","longitude","timestamp","description","status"]]

In [33]:
# add metadata
weihnacht = pd.merge(weihnacht, metadata, on='shelter_id')

In [34]:
weihnacht.to_csv("./data/cleaned/belegungsstatus_25-12-2022_8-01.csv", index=False)